In [1]:
%gui qt
import sys
from pathlib import Path
import urllib3

from dataclasses import dataclass
from scipy.spatial import ConvexHull

import numpy as np

import meshio
import zarr
import napari

sys.path.append("..")
import ply_zarr

from ome_zarr.utils import download

## Load data from IDR and download it

In [2]:
idr_data = "https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/"


idr_name = idr_data.rstrip('/').split("/")[-1]

local_path = Path("../data/") / idr_name
if not local_path.exists():
    download(idr_data, local_path.as_posix())


In [3]:
data = zarr.open(local_path.as_posix(), "a")

In [4]:
data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [5]:
labels = data['labels']["masks"][0][0]

lbls = np.unique(labels)

meshes = []
for idx in lbls[1:6]:
    _, x, y, z = (labels == idx).nonzero()
    points = np.vstack([x, y, z]).T
    hull = ConvexHull(points)
    triangles = hull.simplices
    color = (points * [0.1, 0.3, 0.2]).sum(axis=1)
    
    mesh = meshio.Mesh(
        points=points,
        cells=[("triangle", triangles),],
        point_data={"color": color})
    
    mesh.remove_orphaned_nodes()
    meshes.append(mesh)

In [6]:
!rm -Rf {local_path / "meshes"}

In [7]:
mesh_grp = data.create_group('meshes')

In [8]:
for i, mesh in enumerate(meshes):
    grp = mesh_grp.create_group(i)
    ply_zarr.write(grp, mesh)


In [9]:
data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [10]:
print(mesh_grp[0].tree())

0
 ├── 3
 │   └── vertex_indices (336, 3) int32
 └── points
     ├── color (170,) float64
     ├── x (170,) int64
     ├── y (170,) int64
     └── z (170,) int64


In [20]:
mesh_grp[0].attrs.asdict()

{'comments': ['Created by ply-zarr v0.0.1, 2021-05-06T12:00:52.642795'],
 'elements': {'face': {'properties': [['list',
     'uint8',
     'int32',
     'vertex_indices']],
   'size': 336},
  'vertex': {'properties': [['int64', 'x'],
    ['int64', 'y'],
    ['int64', 'z'],
    ['double', 'color']],
   'size': 170}},
 'format': 'ascii 1.0'}

In [11]:
mesh_reloaded = ply_zarr.read(mesh_grp[0])

In [12]:
viewer = napari.Viewer()
viewer.add_image(data[0])

for mesh in meshes:
    viewer.add_surface(
        (mesh.points,
         mesh.cells[0].data,
         mesh.point_data["color"]),
        colormap="Blues", opacity=1
    )

surf = viewer.add_surface(
        (mesh_reloaded.points,
         mesh_reloaded.cells[0].data,
         mesh_reloaded.point_data["color"]),
        colormap="Reds", opacity=1
    )

<Surface layer 'Surface [5]' at 0x7fa05c605550>